In [2]:
using Pkg
using XSim, JWAS, CSV, DelimitedFiles, DataFrames, Statistics, StatsBase, LinearAlgebra, XSimPreProcess

┌ Info: Precompiling XSimPreProcess [top-level]
└ @ Base loading.jl:1187


In [ ]:
# function generate_XSim_data()
#     for i in 1:3
#numChr,numLoci,chrLength,mutRate = 2,[1,2],[0.1,0.2],0.0
chrLength= 1.0  #length of each chromosome
numChr   = 2    #number of chromosomes
nmarkers = 1500   #number of loci for each chromosome
nQTL     = 25    #number of QTL for each chromosomefects,mutRate);
mutRate = 0.000025
#geneFreq = fill(0.5,nmarkers)
build_genome(numChr,chrLength,nmarkers,nQTL)

# generate 4 generations

#generation 0
#generate founders
popSizeFounder = 100
sires = sampleFounders(popSizeFounder);
dams  = sampleFounders(popSizeFounder);

#random mating
ngen,popSize = 1,2800
sires1,dams1,gen1 = sampleRan(popSize, ngen, sires, dams);


ngen, popSize=1, 2800
sires2,dams2,gen2 = sampleRan(popSize, ngen, sires1, dams1);

ngen, popSize=1, 2800
sires3,dams3,gen3 = sampleRan(popSize, ngen, sires2, dams2);

ngen, popSize= 1, 2800
sires4,dams4,gen4 = sampleRan(popSize, ngen, sires3, dams3);


ngen, popSize = 1, 2800
sires5, dams5, gen5 = sampleRan(popSize, ngen, sires4, dams4);

# #collect the animals data
animals5 = concatCohorts(sires5,dams5);
animals4 = concatCohorts(sires4, dams4);
animals3 = concatCohorts(sires3, dams3);
animals2 = concatCohorts(sires2, dams2);
animals1 = concatCohorts(sires1, dams1);
animals0 = concatCohorts(sires, dams);

#get genotypes
M5 = getOurGenotypes(animals5);
M4 = getOurGenotypes(animals4);
M3 = getOurGenotypes(animals3);

M = vcat(M3, M4, M5);

#get phenotypes
Phen1 = getOurPhenVals(animals1, 0.1);
var1 = var(Phen1)
Phen2 = getOurPhenVals(animals2, 0.5*var1);
Phen3 = getOurPhenVals(animals3, 0.5*var1);
Phen4 = getOurPhenVals(animals4, 0.5*var1);
Phen5 = getOurPhenVals(animals5, 0.5*var1);
Phen = vcat(Phen1, Phen2, Phen3, Phen4, Phen5);


#get pedigree
Ped5 = getPedigree(animals5);
Ped4 = getPedigree(animals4);
Ped3 = getPedigree(animals3);
Ped2 = getPedigree(animals2);
Ped1 = getPedigree(animals1)
Ped = vcat(Ped1, Ped2, Ped3, Ped4, Ped5);

#write out
Phenotypes = DataFrame();
Phenotypes[:y] = Phen;

In [ ]:
f_pheno = open("../data\\Phenotypes_XSim_Raw.txt", "w")
f_geno = open("../data\\Genotypes_XSim_Raw.txt", "w")
f_ped = open("../data\\Pedigree_XSim_Raw.txt", "w")
outputCatData("../data\\map_XSim_Raw.txt")        
CSV.write(f_pheno, Phenotypes)
writedlm(f_geno, M)
writedlm(f_ped, Ped)
close(f_pheno)
close(f_geno)
close(f_ped)

In [5]:
#Phenotypes = readdlm("/home/adhal/home/Projects/SSBR3/XSim-Data$i/Phenotypes_XSim_Raw$i.txt");
#M = readdlm("/home/adhal/home/Projects/SSBR3/XSim-Data$i/Genotypes_XSim_Raw$i.txt");
#Ped = readdlm("/home/adhal/home/Projects/SSBR3/XSim-Data$i/Pedigree_XSim_Raw$i.txt");

## Making training and test
## Make sure to conserve order in the concatenation

## pheno_setup
## pheno training setup

pheno_train_ng = Phenotypes[1:3920, :];
pheno_train_g = Phenotypes[5601:11480, :];
pheno_train = vcat(pheno_train_ng, pheno_train_g);
CSV.write("../Processed-Data\\pheno_train.txt", pheno_train)

## pheno testing setup
pheno_test_ng = Phenotypes[3921:5600, :];
pheno_test_g = Phenotypes[11481:14000, :];
pheno_test = vcat(pheno_test_ng, pheno_test_g)

## geno_setup
geno_train_g = M[1:5880, :]
geno_test_g = M[5881:8400, :];

## Split pedigree file for extracting rowIDs for the simualtion
## ped_training setup
ped_train_ng = Ped[1:3920, :];
ped_train_g = Ped[5601:11480, :];
ped_train = vcat(ped_train_ng, ped_train_g);

## ped_testing setup
ped_test_ng = Ped[3921:5600, :];
ped_test_g = Ped[11481:14000, :];
ped_test = vcat(ped_test_ng, ped_test_g);

## preprocess and save pedigree file
ped_out_1 = ped_ssbr_convert_1(Ped);
ped_out = ped_ssbr_append_char(ped_out_1);
ped_out_process = open("../Processed-Data\\pedigree_processed.txt", "w");
writedlm(ped_out_process, ped_out);
close(ped_out_process)

## get ObsID
obs_ID = pheno_row_IDs_test(pheno_test, ped_test);
println(obs_ID)
obs_out = open("../Processed-Data\\Obs_ID.txt", "w")
writedlm(obs_out, obs_ID)
close(obs_out)

## get genotype pre-processed
geno_train_IDs = geno_row_IDs(geno_train_g, ped_train_g);
geno_out = geno_ssbr_format(geno_train_g, geno_train_IDs, 3000);
geno_name = open("../Processed-Data\\geno_processed.txt", "w");
writedlm(geno_name, geno_out)
close(geno_name)

## get pheno-preprocessed
pheno_train = readdlm("../Processed-Data\\pheno_train.txt")
## pheno_train IDs
pheno_train_IDs = pheno_row_IDs_train(pheno_train, ped_train);

## phenotype test IDs
pheno_test_IDs = pheno_row_IDs_test(pheno_test, ped_test);
println(pheno_test_IDs)
pheno_train_processed = pheno_ssbr_format(pheno_train, pheno_train_IDs);
## write out pheno_train to output
pheno_train_process_out = open("../Processed-Data\\pheno_processed.txt", "w");
writedlm(pheno_train_process_out, pheno_train_processed);
close(pheno_train_process_out)
 

# end
   
# end

# generate_XSim_data()

  0.572570 seconds (1.98 M allocations: 99.951 MiB, 48.12% gc time)


Any["a4121", "a4122", "a4123", "a4124", "a4125", "a4126", "a4127", "a4128", "a4129", "a4130", "a4131", "a4132", "a4133", "a4134", "a4135", "a4136", "a4137", "a4138", "a4139", "a4140", "a4141", "a4142", "a4143", "a4144", "a4145", "a4146", "a4147", "a4148", "a4149", "a4150", "a4151", "a4152", "a4153", "a4154", "a4155", "a4156", "a4157", "a4158", "a4159", "a4160", "a4161", "a4162", "a4163", "a4164", "a4165", "a4166", "a4167", "a4168", "a4169", "a4170", "a4171", "a4172", "a4173", "a4174", "a4175", "a4176", "a4177", "a4178", "a4179", "a4180", "a4181", "a4182", "a4183", "a4184", "a4185", "a4186", "a4187", "a4188", "a4189", "a4190", "a4191", "a4192", "a4193", "a4194", "a4195", "a4196", "a4197", "a4198", "a4199", "a4200", "a4201", "a4202", "a4203", "a4204", "a4205", "a4206", "a4207", "a4208", "a4209", "a4210", "a4211", "a4212", "a4213", "a4214", "a4215", "a4216", "a4217", "a4218", "a4219", "a4220", "a4221", "a4222", "a4223", "a4224", "a4225", "a4226", "a4227", "a4228", "a4229", "a4230", "a4231

, "a12689", "a12690", "a12691", "a12692", "a12693", "a12694", "a12695", "a12696", "a12697", "a12698", "a12699", "a12700", "a12701", "a12702", "a12703", "a12704", "a12705", "a12706", "a12707", "a12708", "a12709", "a12710", "a12711", "a12712", "a12713", "a12714", "a12715", "a12716", "a12717", "a12718", "a12719", "a12720", "a12721", "a12722", "a12723", "a12724", "a12725", "a12726", "a12727", "a12728", "a12729", "a12730", "a12731", "a12732", "a12733", "a12734", "a12735", "a12736", "a12737", "a12738", "a12739", "a12740", "a12741", "a12742", "a12743", "a12744", "a12745", "a12746", "a12747", "a12748", "a12749", "a12750", "a12751", "a12752", "a12753", "a12754", "a12755", "a12756", "a12757", "a12758", "a12759", "a12760", "a12761", "a12762", "a12763", "a12764", "a12765", "a12766", "a12767", "a12768", "a12769", "a12770", "a12771", "a12772", "a12773", "a12774", "a12775", "a12776", "a12777", "a12778", "a12779", "a12780", "a12781", "a12782", "a12783", "a12784", "a12785", "a12786", "a12787", "a12788"

3519", "a13520", "a13521", "a13522", "a13523", "a13524", "a13525", "a13526", "a13527", "a13528", "a13529", "a13530", "a13531", "a13532", "a13533", "a13534", "a13535", "a13536", "a13537", "a13538", "a13539", "a13540", "a13541", "a13542", "a13543", "a13544", "a13545", "a13546", "a13547", "a13548", "a13549", "a13550", "a13551", "a13552", "a13553", "a13554", "a13555", "a13556", "a13557", "a13558", "a13559", "a13560", "a13561", "a13562", "a13563", "a13564", "a13565", "a13566", "a13567", "a13568", "a13569", "a13570", "a13571", "a13572", "a13573", "a13574", "a13575", "a13576", "a13577", "a13578", "a13579", "a13580", "a13581", "a13582", "a13583", "a13584", "a13585", "a13586", "a13587", "a13588", "a13589", "a13590", "a13591", "a13592", "a13593", "a13594", "a13595", "a13596", "a13597", "a13598", "a13599", "a13600", "a13601", "a13602", "a13603", "a13604", "a13605", "a13606", "a13607", "a13608", "a13609", "a13610", "a13611", "a13612", "a13613", "a13614", "a13615", "a13616", "a13617", "a13618", "a1

 95.779429 seconds (311.79 M allocations: 238.680 GiB, 23.19% gc time)
Matrix split intoa1, 0, 1, 2


Any["a4121", "a4122", "a4123", "a4124", "a4125", "a4126", "a4127", "a4128", "a4129", "a4130", "a4131", "a4132", "a4133", "a4134", "a4135", "a4136", "a4137", "a4138", "a4139", "a4140", "a4141", "a4142", "a4143", "a4144", "a4145", "a4146", "a4147", "a4148", "a4149", "a4150", "a4151", "a4152", "a4153", "a4154", "a4155", "a4156", "a4157", "a4158", "a4159", "a4160", "a4161", "a4162", "a4163", "a4164", "a4165", "a4166", "a4167", "a4168", "a4169", "a4170", "a4171", "a4172", "a4173", "a4174", "a4175", "a4176", "a4177", "a4178", "a4179", "a4180", "a4181", "a4182", "a4183", "a4184", "a4185", "a4186", "a4187", "a4188", "a4189", "a4190", "a4191", "a4192", "a4193", "a4194", "a4195", "a4196", "a4197", "a4198", "a4199", "a4200", "a4201", "a4202", "a4203", "a4204", "a4205", "a4206", "a4207", "a4208", "a4209", "a4210", "a4211", "a4212", "a4213", "a4214", "a4215", "a4216", "a4217", "a4218", "a4219", "a4220", "a4221", "a4222", "a4223", "a4224", "a4225", "a4226", "a4227", "a4228", "a4229", "a4230", "a4231

", "a12862", "a12863", "a12864", "a12865", "a12866", "a12867", "a12868", "a12869", "a12870", "a12871", "a12872", "a12873", "a12874", "a12875", "a12876", "a12877", "a12878", "a12879", "a12880", "a12881", "a12882", "a12883", "a12884", "a12885", "a12886", "a12887", "a12888", "a12889", "a12890", "a12891", "a12892", "a12893", "a12894", "a12895", "a12896", "a12897", "a12898", "a12899", "a12900", "a12901", "a12902", "a12903", "a12904", "a12905", "a12906", "a12907", "a12908", "a12909", "a12910", "a12911", "a12912", "a12913", "a12914", "a12915", "a12916", "a12917", "a12918", "a12919", "a12920", "a12921", "a12922", "a12923", "a12924", "a12925", "a12926", "a12927", "a12928", "a12929", "a12930", "a12931", "a12932", "a12933", "a12934", "a12935", "a12936", "a12937", "a12938", "a12939", "a12940", "a12941", "a12942", "a12943", "a12944", "a12945", "a12946", "a12947", "a12948", "a12949", "a12950", "a12951", "a12952", "a12953", "a12954", "a12955", "a12956", "a12957", "a12958", "a12959", "a12960", "a12961

a13734", "a13735", "a13736", "a13737", "a13738", "a13739", "a13740", "a13741", "a13742", "a13743", "a13744", "a13745", "a13746", "a13747", "a13748", "a13749", "a13750", "a13751", "a13752", "a13753", "a13754", "a13755", "a13756", "a13757", "a13758", "a13759", "a13760", "a13761", "a13762", "a13763", "a13764", "a13765", "a13766", "a13767", "a13768", "a13769", "a13770", "a13771", "a13772", "a13773", "a13774", "a13775", "a13776", "a13777", "a13778", "a13779", "a13780", "a13781", "a13782", "a13783", "a13784", "a13785", "a13786", "a13787", "a13788", "a13789", "a13790", "a13791", "a13792", "a13793", "a13794", "a13795", "a13796", "a13797", "a13798", "a13799", "a13800", "a13801", "a13802", "a13803", "a13804", "a13805", "a13806", "a13807", "a13808", "a13809", "a13810", "a13811", "a13812", "a13813", "a13814", "a13815", "a13816", "a13817", "a13818", "a13819", "a13820", "a13821", "a13822", "a13823", "a13824", "a13825", "a13826", "a13827", "a13828", "a13829", "a13830", "a13831", "a13832", "a13833", "